In [1]:
from PyQt5.QtCore import *
from PyQt5.QtGui import *
from PyQt5.QtWidgets import *

import matplotlib.pyplot as plt
import numpy as np
import scipy as sc
import random
from matplotlib.backends.backend_qt5agg import (NavigationToolbar2QT as NavigationToolbar)
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas


import sys
import time
import os

In [2]:
%run "UI/main_window.ipynb"
%run "UI/dialog_topology_config.ipynb"

In [3]:
%run "DAL/file_manager.ipynb"

In [4]:
#%run "BLL/widget_graphics.ipynb"
%run "BLL/network_logic.ipynb"

## Window: *Topology Editor*

In [5]:
class TopologyEditor(QDialog, Ui_Dialog):
    def __init__(self, parent):
        super().__init__(parent)
        self.setup_ui(self)
        #self.setModal(True)
        self.parent = parent

        self.setup_data()
        self.setup_buttons()
        self.setup_topology()
        self.show_hidden_layers()
        self.tableWidget_hidden_layers.horizontalHeader().setStretchLastSection(True)

        
    def setup_data(self):
        
        activation_functions_hidden = ActivationFunction().names_activation_functions_hidden
        activation_functions_final = ActivationFunction().names_activation_functions_final
                
        self.comboBox_activation_function_hidden.addItems(activation_functions_hidden)
        self.comboBox_activation_function_final.addItems(activation_functions_final)
        
        self.tableWidget_hidden_layers.setRowCount(1)
        self.tableWidget_hidden_layers.setColumnCount(2)
        self.tableWidget_hidden_layers.setItem(0, 0, QTableWidgetItem("Número de neuronas"))
        self.tableWidget_hidden_layers.setItem(0, 1, QTableWidgetItem("Funcion de activación"))
             
        
    
    def setup_buttons(self):
        
        self.pushButton_add_hidden_layer.clicked.connect(self.add_hidden_layer)
        
        self.pushButton_save_topology.clicked.connect(self.save_topology)
        self.pushButton_reset_topology.clicked.connect(self.reset_topology)
        
    
    def setup_topology(self):
        
        try:
            self.topology = self.parent.topology.copy()
            self.activation_functions = self.parent.activation_functions.copy()
            
            self.activation_functions.del(-1)
            
            self.comboBox_activation_function_final.setCurrentIndex(self.parent.activation_functions[-1])
        
        except:
            
            self.topology = [self.parent.dataset.n_X, self.parent.dataset.n_Y]
        
            self.activation_functions = [-1]            
        
    
    def add_hidden_layer(self):
        
        self.topology.insert(-1, int(self.lineEdit_neurons_number_hidden.text()))
        
        self.activation_functions.append(self.comboBox_activation_function_hidden.currentIndex())
        
        self.show_hidden_layers()
        
        
    def show_hidden_layers(self):
        
        _nx = len(self.topology)
        
        self.tableWidget_hidden_layers.setRowCount(_nx-1)
        
        for i_x in range(1, _nx-1):
            
            self.tableWidget_hidden_layers.setItem(i_x, 0, QTableWidgetItem(str(self.topology[i_x])))
            print(str(self.topology[i_x]))
            
        for i_x in range(1, _nx-1):
            
            self.tableWidget_hidden_layers.setItem(i_x, 1, QTableWidgetItem(ActivationFunction().names_activation_functions_hidden[self.activation_functions[i_x]]))
            
        self.parent.draw_nn(self.topology)

    
    def save_topology(self):
        
        self.activation_functions.append(self.comboBox_activation_function_final.currentIndex())
        
        self.parent.topology = self.topology
        self.parent.activation_functions = self.activation_functions
        
        self.close()
        
    def reset_topology(self):
        
        self.topology = [self.parent.dataset.n_X, self.parent.dataset.n_Y]
        
        self.activation_functions = [-1]
        
        self.show_hidden_layers()
        

SyntaxError: invalid syntax (<ipython-input-5-7e3b63c1caf8>, line 44)

In [ ]:
class hol():
    def __init__(self, parent):
        self.parent = parent
        self.t()
    def t(self):
        self.parent.et = 6
class hel():
    def hola1(self):
        hol(self)
        print (self.et)
        
hel().hola1()

## Window: *Main*

In [ ]:
class MainWindow(QMainWindow, Ui_MainWindow):
    def __init__(self):
        super().__init__()
        self.setup_ui(self, "UI")
        self.setup_actions()
        
        
    def setup_actions(self):
        
        self.action_new.triggered.connect(self.new_project)
        
    def setup_buttons(self):
        self.pushButton_upload_dataset.clicked.connect(self.upload_dataset)
        self.pushButton_open_topology_editor.clicked.connect(self.open_topology_editor)
        
        self.pushButton_execute.clicked.connect(self.execute)
        
        
    def new_project(self):
        
        try:
            project_path, _format = QFileDialog.getSaveFileName(self, 'New project', '/home/earodriguezm/', "NeuNet (.neunet)")
        except:
            QMessageBox.about(self, "Error", "Error en la creación del proyecto")
            return False
        
        self.file_manager = FileManager(project_path)
        
        self.setWindowTitle(self.file_manager.project_name)
        
        self.create_q_objects(self)
        self.setup_buttons()
        self.tableWidget_dataset.horizontalHeader().setStretchLastSection(True)

        
        
    def upload_dataset(self):

        try:
            dataset_path, _format = QFileDialog.getOpenFileName(self, 'Upload dataset file', self.file_manager.project_path, "CSV (*.csv) ;; TSV (*.tsv)")
        except:
            QMessageBox.about(self, "Error", "Error en la creación del proyecto")
            return False
        
        if _format == "CSV (*.csv)":
            delimiter_format = ','           
        if _format == "TSV (*.tsv)":
            delimiter_format = '\t'
        
        self.lineEdit_dataset_filename.setText(dataset_path)
        
        self.file_manager.dataset_path = dataset_path
        self.file_manager.delimiter_format = delimiter_format
        self.show_dataset()
        
        self.dataset = Dataset(self.file_manager.count_in_out())
        self.draw_nn([self.dataset.n_X, self.dataset.n_Y])
        
        self.dataset.import_X_dataset(self.file_manager)
        self.dataset.import_Y_dataset(self.file_manager)
        self.set_default_config()
        
        
    def show_dataset(self):
        
        _data = self.file_manager.import_file(self.file_manager.dataset_path)
        
        _x, _y = _data.shape
        self.tableWidget_dataset.setRowCount(_x)
        self.tableWidget_dataset.setColumnCount(_y)
        
        for i_x in range(0, _x):
            for i_y in range(0, _y):
                self.tableWidget_dataset.setItem(i_x, i_y, QTableWidgetItem(_data[i_x, i_y]))
                
    def draw_nn(self, topology):
        
        widest_layer = max(topology)
        self.graphic_neural_network = GraphicNeuralNetwork(widest_layer)
        for l in topology:
            self.graphic_neural_network.add_layer(l)
        self.graphic_neural_network.draw()
        
    def set_default_config(self):

        self.lineEdit_learning_ratio.setText("0.01")
        self.lineEdit_iterations.setText("1000")
        self.lineEdit_err_max.setText("0.1")
        self.lineEdit_res.setText("50")
        self.lineEdit_interval_show.setText("2")
        
        self.pushButton_open_topology_editor.setEnabled(True)
        
        self.set_backpropagation_options(True)
        
    def open_topology_editor(self):

        topology_editor = TopologyEditor(self)
        topology_editor.exec()
        
        self.draw_nn(self.topology)

        
        
        
    def set_backpropagation_options(self, status = False):
        
        self.radioButton_backpropagation_primitive.setEnabled(status)
        self.radioButton_backpropagation_primitive.toggled.connect(lambda:self.set_execute_button_status(True))
        
        self.radioButton_backpropagation_cascade.setEnabled(status)
        self.radioButton_backpropagation_cascade.toggled.connect(lambda:self.set_execute_button_status(True))
        
        self.radioButton_backpropagation_without.setEnabled(status)
        self.radioButton_backpropagation_without.toggled.connect(lambda:self.set_execute_button_status(True)) 
        
    def set_execute_button_status(self, status = True):
        
        self.pushButton_execute.setEnabled(True)
        
    def execute(self):
        print(self.some_value_of_test)


In [ ]:
app = QApplication(sys.argv)
main_window = MainWindow()
main_window.show()
app.exec_()